На Кинопоиск можно импортировать оценки с IMDB. Мне же потребовалось перенести оценки и любимые фильмы с одного аккаунта Кинопоиска на другой. Необходимо иметь установленный Chromium/Google Chrome и ChromeDriver. Вход и ввод капчи производятся вручную.

# Получение оценок из старого аккаунта

In [1]:
import math
import time
from selenium import webdriver
path_to_chromedriver = '/Applications/chromedriver'
driver = webdriver.Chrome(path_to_chromedriver) 
driver.get('https://www.kinopoisk.ru/#login')

# вхожу в аккаунт, из которого хочу перенести оценки и любимые фильмы
# выделяю 30 секунд для авторизации и ввода капчи вручную. 

time.sleep(30)

# после входа в систему определяю user_id

user_icon_selector = 'div.header-fresh-partial-component__user-container > div > div.header-fresh-user-partial-component__button > div > span'
user_id_selector = 'div.header-fresh-user-partial-component__dropdown.header-fresh-user-partial-component__dropdown_visible > div > ul > div:nth-child(1) > li > a'

driver.find_element_by_css_selector(user_icon_selector).click()
driver.find_element_by_css_selector(user_id_selector).click()
user_id = driver.current_url.split('user/')[1].split('/')[0]

# захожу на первую страницу с оценками, считаю количество страниц, по которым надо будет итерироваться
ratings_page = 'https://www.kinopoisk.ru/user/' + user_id +'/votes/list/ord/date/page/{}/#list'
driver.get(ratings_page.format(1))
num_movies_selector = 'tbody > tr > td > h2'
num_movies = driver.find_element_by_css_selector(num_movies_selector).text.split()[2][1:-1]
num_pages = math.ceil(int(num_movies) / 50)

# скачиваю просмотренные фильмы и выставленные им оценки

movie_link_selector = 'div.nameRus > a'
movie_rating_selector = 'div.selects.vote_widget span'
all_links = []
all_ratings = []

for i in range(1, num_pages+1):
    driver.get(ratings_page.format(i))
    movie_links = driver.find_elements_by_css_selector(movie_link_selector)
    ratings = driver.find_elements_by_css_selector(movie_rating_selector)
    all_links.extend([link.get_attribute('href') for link in movie_links])
    all_ratings.extend([item.text for item in ratings])

# захожу на первую страницу с любимыми фильмами, считаю количество страниц

faves_page = 'https://www.kinopoisk.ru/user/' + user_id +'/movies/list/sort/default/vector/desc/page/{}/#list'
driver.get(faves_page.format(1))
num_movies = driver.find_element_by_class_name('num').text[1:-1]
num_pages = math.ceil(int(num_movies) / 25)

# скачиваю любимые фильмы

movie_link_selector = 'div.info > div > font > a'
faves = []

for i in range(1, num_fav_pages+1):
    driver.get(faves_page.format(i))
    movie_links = driver.find_elements_by_css_selector(movie_link_selector)
    faves.extend([link.get_attribute('href') for link in movie_links])    
all_ratings = [x for x in all_ratings if x != '']

driver.quit()

# Проставление полученных оценок в новом аккаунте

In [2]:
driver = webdriver.Chrome('/Applications/chromedriver')
url = 'https://www.kinopoisk.ru/'
driver.get(url)

# выделяю 30 секунд для авторизации и ввода капчи вручную

time.sleep(30)

# вхожу в аккаунт, в который хочу перенести оценки и любимые фильмы
# проставляю оценки all_ratings фильмам по ссылкам из списка all_links

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions

for page, rating in zip(reversed(all_links), reversed(all_ratings)):
    driver.get(page)
    try:
        star = WebDriverWait(driver, 7).until(
                expected_conditions.presence_of_element_located((By.CLASS_NAME, 's{}'.format(rating)))).click()
    except:
        print('Failed to rate movie {}'.format(page))
        continue
                
# проставляю любимые фильмы по ссылкам из списка faves

faves_selector = 'a#btn_fav_film'
for page in reversed(faves):
    driver.get(page)
    try:
        star = WebDriverWait(driver, 7).until(
                expected_conditions.presence_of_element_located((By.CSS_SELECTOR, faves_selector))).click()   
    except:
        print('Failed to add movie to favourites: {}'.format(page))
        continue           